In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movie = pd.read_csv('movies.csv')

In [3]:
movie.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movie.shape

(4803, 24)

In [5]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [6]:
movie.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [7]:
selected_features = ['genres','keywords','tagline','cast','director']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [8]:
for feature in selected_features:
    movie[feature] = movie[feature].fillna('')

In [9]:
combined_features = movie['genres'] + ' ' + movie['keywords'] + ' ' + movie['tagline'] + ' ' + movie['cast'] + ' ' + movie['director']
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [10]:
vectorizer = TfidfVectorizer()

In [11]:
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

Cosine Similarity

In [12]:
similarity = cosine_similarity(feature_vectors)

In [13]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [14]:
print(similarity.shape)

(4803, 4803)


Input movie name from the user

In [15]:
movie_name = input("Enter Movie Name: ")

Enter Movie Name: Iron man 3


In [16]:
list_titles = movie['title'].tolist()
print(list_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [17]:
find_close_match = difflib.get_close_matches(movie_name, list_titles)
print(find_close_match)

['Iron Man 3', 'Iron Man 2', 'Iron Man']


In [18]:
close_match = find_close_match[0]
print(close_match)

Iron Man 3


In [19]:
index_of_movie = movie[movie.title == close_match]['index'].values[0]
print(index_of_movie)

31


In [20]:
similarity_score = list(enumerate(similarity[index_of_movie]))
print(similarity_score)

[(0, 0.060555221380551486), (1, 0.045173032380404654), (2, 0.025587599816574425), (3, 0.0596552556152424), (4, 0.04491082754059965), (5, 0.024826539308291363), (6, 0.008095033555461723), (7, 0.22661385170966206), (8, 0.00826062907345574), (9, 0.05587767668577865), (10, 0.05070248377744623), (11, 0.012498348966608383), (12, 0.014365386614171867), (13, 0.022093150184307138), (14, 0.08709976256786348), (15, 0.018080853082791858), (16, 0.21487810271500662), (17, 0.013742159990800978), (18, 0.03542729148083272), (19, 0.02936516897159686), (20, 0.09349691478373992), (21, 0.011807521103905674), (22, 0.016161416228396827), (23, 0.024030803535979178), (24, 0.0550631717498923), (25, 0.011014776597901378), (26, 0.22176049914086776), (27, 0.0410639058734783), (28, 0.045083322709864135), (29, 0.02616636641966004), (30, 0.08464943031376658), (31, 1.0000000000000002), (32, 0.01752083358456192), (33, 0.14815320951899616), (34, 0.0), (35, 0.03704638648420709), (36, 0.04806705555546236), (37, 0.00840937

In [21]:
len(similarity_score)

4803

In [22]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(31, 1.0000000000000002), (79, 0.49790727997578726), (68, 0.31467052449477506), (7, 0.22661385170966206), (26, 0.22176049914086776), (16, 0.21487810271500662), (182, 0.16398518362587788), (46, 0.1610001695338198), (2625, 0.15868163752956246), (4401, 0.15202049419143115), (511, 0.15049377529939592), (356, 0.15008507803087162), (33, 0.14815320951899616), (64, 0.1459070770911358), (203, 0.1424625208947325), (466, 0.13913012151683266), (205, 0.13575975331420798), (607, 0.1346030776822172), (1439, 0.13406677583947127), (174, 0.1326265625970645), (85, 0.13136489442315133), (38, 0.1291873755087727), (2487, 0.12900421986389907), (788, 0.1263043405929412), (126, 0.1259064940300999), (122, 0.12441640722844889), (101, 0.12385386970885616), (1180, 0.12209689666367265), (169, 0.11887983807528442), (505, 0.11799432353354633), (2651, 0.1175712829925276), (720, 0.11550329137310283), (2235, 0.11452060089181132), (2357, 0.1138883499332945), (242, 0.1114044236834965), (1651, 0.1097583658535816), (1931, 

In [23]:
print("Movies suggested for you: \n")
i = 1
for movie_sugg in sorted_similar_movies:
    index = movie_sugg[0]
    title = movie[movie.index==index]['title'].values[0]
    if (i < 30):
        print(i, ')', title)
        i+=1

Movies suggested for you: 

1 ) Iron Man 3
2 ) Iron Man 2
3 ) Iron Man
4 ) Avengers: Age of Ultron
5 ) Captain America: Civil War
6 ) The Avengers
7 ) Ant-Man
8 ) X-Men: Days of Future Past
9 ) Kiss Kiss Bang Bang
10 ) The Helix... Loaded
11 ) X-Men
12 ) Sherlock Holmes
13 ) X-Men: The Last Stand
14 ) X-Men: Apocalypse
15 ) X2
16 ) The Time Machine
17 ) Sherlock Holmes: A Game of Shadows
18 ) Sky Captain and the World of Tomorrow
19 ) Lions for Lambs
20 ) The Incredible Hulk
21 ) Captain America: The Winter Soldier
22 ) The Amazing Spider-Man 2
23 ) Duets
24 ) Deadpool
25 ) Thor: The Dark World
26 ) X-Men Origins: Wolverine
27 ) X-Men: First Class
28 ) Shallow Hal
29 ) Captain America: The First Avenger


In [25]:
movie_name = input('Enter Movie Name: ')

list_of_all_titles = movie['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movie[movie.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movie[movie.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Enter Movie Name: batman


TypeError: tuple indices must be integers or slices, not str